In [45]:
from typing import DefaultDict
from typing import List

strs = ["act","pots","tops","cat","stop","hat"]

In [2]:
sorted(strs)

['act', 'cat', 'hat', 'pots', 'stop', 'tops']

In [5]:
for i in range(len(strs)):
    strs[i] = sorted(strs[i])

In [6]:
strs

[['a', 'c', 't'],
 ['o', 'p', 's', 't'],
 ['o', 'p', 's', 't'],
 ['a', 'c', 't'],
 ['o', 'p', 's', 't'],
 ['a', 'h', 't']]

In [18]:
list = []
for s in strs:
    sortedS = ''.join(sorted(s))
    list.append(sortedS)

In [19]:
sortedS

'aht'

In [20]:
list

['act', 'opst', 'opst', 'act', 'opst', 'aht']

In [22]:
class Solution:
    def groupAnagrams(self, strs: List[str]) -> List[List[str]]:
        ans = DefaultDict(list)

        for s in strs:
            key = "".join(sorted(s))
            ans[key].append(s)

        return list(ans.values())

NameError: name 'List' is not defined

In [27]:
new_list = []

for s in strs:
    key = "".join(sorted(s))
    new_list.append(key)

In [26]:
key

'aht'

In [28]:
new_list

['act', 'opst', 'opst', 'act', 'opst', 'aht']

In [29]:
new_list = []

for s in strs:
    key = sorted(s)
    new_list.append(key)

In [30]:
new_list

[['a', 'c', 't'],
 ['o', 'p', 's', 't'],
 ['o', 'p', 's', 't'],
 ['a', 'c', 't'],
 ['o', 'p', 's', 't'],
 ['a', 'h', 't']]

In [32]:
print(strs.sort())

None


In [34]:
a = [1, 11, 2]
x = sorted(a)
print(x)

[1, 2, 11]


In [36]:
a = "string"
x = sorted(a)
print(x)

['g', 'i', 'n', 'r', 's', 't']


In [37]:
strs = ["act","pots","tops","cat","stop","hat"]

In [38]:
strs[0]

'act'

In [40]:
map_T = {}
map_T[strs[0]] = 1 + map_T.get(strs[0], 0)

In [41]:
map_T

{'act': 1}

In [42]:
strs[0][0]

'a'

In [46]:
res = DefaultDict(list)

TypeError: first argument must be callable or None

In [47]:
res = {}

In [50]:
for s in strs:
    count = [0] * 26
    for c in s:
        count[ord(c) - ord('a')] += 1
    res[tuple(count)].append(s)

KeyError: (1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0)